# Alocação e otimização de portfólios

## Importando libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

## Importando os dados

In [ ]:
dataset = pd.read_csv('acoes.csv')
dataset

## Alocação aleatório de ativos

In [ ]:
def alocacao_ativos(dataset, dinheiro_total, seed = 0, melhores_pesos = []):
    dataset = dataset.copy()

    if seed != 0  :
        np.random.seed(seed)

    if len(melhores_pesos) > 0:
        pesos = melhores_pesos
    else:
        pesos = np.random.random(len(dataset.columns) - 1)
        #print(pesos, pesos.sum())
        pesos = pesos / pesos.sum()
        #print(pesos, pesos.sum())        

    pesos = np.random.random(len(dataset.columns) - 1)
    pesos = pesos / pesos.sum()
    print(pesos, pesos.sum())

    colunas = dataset.columns[1:]

    for i in colunas:
        dataset[i] = ( dataset[i] / dataset[i][0])

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset['soma valor'] = dataset.iloc[:, 1:].sum(axis = 1)

    datas = dataset['Date']

    dataset.drop(labels= ['Date'], axis = 1, inplace= True)
    dataset['taxa retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa retorno'][i] = ((dataset['soma valor'][i] / dataset['soma valor'][i -1]) - 1) * 100

    acoes_pesos = pd.DataFrame(data = {'Ações': colunas, 'Pesos': pesos * 100})

    return dataset, datas, acoes_pesos, dataset.loc[len(dataset) - 1]['soma valor']

In [ ]:
dataset, datas, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 5000, 10)

In [ ]:
dataset

In [ ]:
datas

In [ ]:
acoes_pesos

In [ ]:
soma_valor

## Visualização de dados

In [ ]:
figura = px.line(x = datas, y = dataset['taxa retorno'], title = 'Retorno diário do Portfólio')
figura.show()

In [ ]:
figura = px.line(title = 'Evolução do patrimônio')
for i in dataset.drop(columns= ['soma valor', 'taxa retorno']).columns:
    figura.add_scatter(x = datas, y = dataset[i], name = i)

figura.show()

In [ ]:
figura = px.line(x = datas, y = dataset['soma valor'], title = 'Evoluçào de Patrimônio')
figura.show()

## Retorno acumulado do período

In [ ]:
dataset.loc[len(dataset) - 1]['soma valor'] / dataset.loc[0]['soma valor'] - 1

## Desvio Padrão

In [ ]:
dataset['taxa retorno'].std()

## Sharpe ratio

In [ ]:
(dataset['taxa retorno'].mean() / dataset['taxa retorno'].std()) * np.sqrt(246)

In [ ]:
dinheiro_total = 5000
soma_valor - dinheiro_total

In [ ]:
taxa_selic_2015 = 12.75
taxa_selic_2016 = 14.25
taxa_selic_2017 = 12.25
taxa_selic_2018 = 6.50
taxa_selic_2019 = 5.0
taxa_selic_2020 = 2.0

In [ ]:
valor_2015 = dinheiro_total + (dinheiro_total * taxa_selic_2015 / 100)
valor_2015

In [ ]:
valor_2016 = valor_2015 + (valor_2015 * taxa_selic_2016 / 100)
valor_2016


In [ ]:
valor_2017 = valor_2016 + (valor_2016 * taxa_selic_2017 / 100)
valor_2017


In [ ]:
valor_2018 = valor_2017 + (valor_2017 * taxa_selic_2018 / 100)
valor_2018


In [ ]:
valor_2019 = valor_2018 + (valor_2018 * taxa_selic_2019 / 100)
valor_2019


In [ ]:
valor_2020 = valor_2019 + (valor_2019 * taxa_selic_2020 / 100)
valor_2020

In [ ]:
redimentos = valor_2020 - dinheiro_total
redimentos

In [ ]:
ir = redimentos * 15 / 100
ir

In [ ]:
valor_2020 - ir

In [ ]:
taxa_selic_historico = np.array([12.75, 14.25, 12.25, 6.5, 5.0, 2.0])

In [ ]:
(dataset['taxa retorno'].mean() - taxa_selic_historico.mean() / 100 / dataset['taxa retorno'].std()) * np.sqrt(246)

## Otimização de portfolio - randômico

In [ ]:
import sys

In [ ]:
def alocacao_portfolio(dataset, dinheiro_total, sem_risco, repeticoes):
    dataset = dataset.copy()
    dataset_original = dataset.copy()

    lista_retorno_esperado = []
    lista_volatilidade_esperada = []
    lista_sharpe_ratio = []

    melhor_sharp_ratio = 1 - sys.maxsize
    melhores_pesos = np.empty
    melhor_volatilidade = 0
    melhor_retorno = 0


    for _ in range(repeticoes):
        pesos = np.random.random(len(dataset.columns) - 1)
        pesos = pesos / pesos.sum()

        colunas = dataset.columns

        for i in colunas[1:]:
            dataset[i] = dataset[i] / dataset[i][0]

        for i, acao in enumerate(dataset.columns[1:]):
            dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

        dataset.drop(labels = ['Date'], axis = 1, inplace=True)

        retorno_carteira = np.log(dataset / dataset.shift(1))
    
        matriz_covariancia = retorno_carteira.cov()

        dataset['soma valor'] = dataset.sum(axis = 1)
        dataset['taxa retorno'] = 0.0

        for i in range(1, len(dataset)):
            dataset['taxa retorno'][i] = np.log(dataset['soma valor'][i] / dataset['soma valor'][i - 1]) * 100

        #sharpe_ratio = (dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std() * np.sqrt(246)       

        retorno_esperado =np.sum(dataset['taxa retorno'].mean() * pesos) * 246
        volatilidade_esperada = np.sqrt(np.dot(pesos, np.dot(matriz_covariancia * 246, pesos)))

        sharpe_ratio = (retorno_esperado - sem_risco) / volatilidade_esperada

        if sharpe_ratio > melhor_sharp_ratio:
            melhor_sharp_ratio = sharpe_ratio
            melhores_pesos = pesos
            melhor_volatilidade = volatilidade_esperada
            melhor_retorno = retorno_esperado

        lista_retorno_esperado.append(retorno_esperado)
        lista_volatilidade_esperada.append(volatilidade_esperada)
        lista_sharpe_ratio.append(sharpe_ratio)

        dataset = dataset_original.copy()

    return melhor_sharp_ratio, melhores_pesos, lista_retorno_esperado, lista_volatilidade_esperada, lista_sharpe_ratio, melhor_volatilidade, melhor_retorno

In [ ]:
sharpe_ratio, melhores_pesos, ls_retono, ls_volatilidade, ls_sharpe_ratio, melhor_volatilidade, melhor_retorno = alocacao_portfolio(pd.read_csv('acoes.csv'), 5000, taxa_selic_historico.mean() / 100, 100)
sharpe_ratio, melhores_pesos

In [ ]:
_, _, acoes_pesos, soma_valor = alocacao_ativos(pd.read_csv('acoes.csv'), 10, melhores_pesos=melhores_pesos)
acoes_pesos, soma_valor

In [ ]:
print(ls_retono)

In [ ]:
print(ls_volatilidade)

In [ ]:
print(ls_sharpe_ratio)

In [ ]:
melhor_retorno, melhor_volatilidade

In [ ]:
plt.Figure(figsize=(10,8))
plt.scatter(ls_volatilidade,ls_retono, c = ls_sharpe_ratio)
plt.colorbar(label = "Sharpe ratio")
plt.xlabel('Volatilidade')
plt.ylabel('Retorno')
plt.scatter(melhor_volatilidade, melhor_retorno, c = 'red', s = 100)

## Otimização de portfólio com algoritmos de otimização

##### SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True pip install mlrose

In [ ]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

In [ ]:
dataset_original = pd.read_csv('acoes.csv')
dinheiro_total  = 5000
sem_risco = taxa_selic_historico.mean() / 100

In [ ]:
def fitness_function(solucao):
    dataset = dataset_original.copy()
    pesos = solucao / solucao.sum()

    colunas = dataset.columns
    for i in colunas[1:]:
        dataset[i] = dataset[i] / dataset[i][0]

    for i, acao in enumerate(dataset.columns[1:]):
        dataset[acao] = dataset[acao] * pesos[i] * dinheiro_total

    dataset.drop(labels = ['Date'], axis = 1, inplace=True)
    dataset['soma valor'] = dataset.sum(axis = 1)
    dataset['taxa retorno'] = 0.0

    for i in range(1, len(dataset)):
        dataset['taxa retorno'][i] = np.log(dataset['soma valor'][i] / dataset['soma valor'][i - 1]) * 100    

    sharpe_ratio = (dataset['taxa retorno'].mean() - sem_risco) / dataset['taxa retorno'].std() * np.sqrt(246)

    return sharpe_ratio

In [ ]:
np.random.seed(10)
pesos = np.random.random(len(dataset_original.columns) - 1)
pesos = pesos / pesos.sum()
pesos


In [ ]:
fitness_function(pesos)

In [ ]:
def visualiza_alocacao(solucao):
    colunas = dataset_original.columns[1:]
    for i in range(len(solucao)):
        print(colunas[i], solucao[i] * 100)

In [ ]:
visualiza_alocacao(pesos)

### Hill climb

### Simulated aneeling

### Algoritmo genético